In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.model_selection import train_test_split

# Assuming 'Training_Essay_Data.csv' is the correct path
pd.set_option('display.max_rows', 5000000)


path = "final_test.csv"
path = "megaset.csv"
path = 'train_v2_drct_02.csv'
path = 'metadataset.csv'
path = 'turboset.csv'
path = 'megaset4.csv'
path = 'train_essays.csv'
data = pd.read_csv(path)


# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.00001, random_state=42)


text_column = 'text'
generated_column = 'generated'

# Define the clean_word function
def clean_word(word):
    word = word.lower()  # Lowercase the word
    cleaned_word = re.sub(r'(?<!\w)[\'\"?!;:,.]|(?<=\s)[\'\"?!;:,.]|[\'\"?!;:,.](?!\w)', '', word)
    return cleaned_word

# Tokenize and clean words in essays
def tokenize_and_clean(text):
    return [clean_word(word) for word in text.lower().split()]

# Ensure essays are strings and not empty or NaN
train_data.dropna(subset=[text_column], inplace=True)
train_data[text_column] = train_data[text_column].astype(str)

# Process AI Essays
ai_essays = train_data[train_data[generated_column] == 1][text_column]
ai_words = []
for essay in ai_essays:
    ai_words.extend(tokenize_and_clean(essay))
ai_word_counts = Counter(ai_words)
ai_total_words = sum(ai_word_counts.values())
ai_word_freq = {word: count / ai_total_words for word, count in ai_word_counts.items()}

# Process Human Essays
human_essays = train_data[train_data[generated_column] == 0][text_column]
human_words = []
for essay in human_essays:
    human_words.extend(tokenize_and_clean(essay))
human_word_counts = Counter(human_words)
human_total_words = sum(human_word_counts.values())
human_word_freq = {word: count / human_total_words for word, count in human_word_counts.items()}

# Prepare the combined DataFrame
word_frequencies_data = []
for word in set(ai_word_freq.keys()).union(set(human_word_freq.keys())):
    ai_freq = ai_word_freq.get(word, 0)
    human_freq = human_word_freq.get(word, 0)
    if human_freq > 0 and ai_freq > 0:  # Ensure word exists in both corpuses
        prevalence_factor = ai_freq / human_freq if ai_freq > human_freq else -human_freq / ai_freq
        word_frequencies_data.append({
            'Word': word,
            'AI_Frequency': ai_freq,
            'Human_Frequency': human_freq,
            'Prevalence_Factor': prevalence_factor
        })


# Create DataFrame
word_frequencies_df = pd.DataFrame(word_frequencies_data)

# Sort by 'Prevalence_Factor'
word_frequencies_df.sort_values(by='Prevalence_Factor', ascending=False, inplace=True)



# drop frequencies column
word_frequencies_df = word_frequencies_df.drop(columns=['AI_Frequency', 'Human_Frequency'])
#write back
word_frequencies_df.to_csv('prevalence_table2.csv', index=False)

#AI essay count
ai_essay_count = len(ai_essays)
#Human essay count
human_essay_count = len(human_essays)

print("AI Essay Count: ", ai_essay_count)
print("Human Essay Count: ", human_essay_count)



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import re

def clean_text(text):
    """Clean and tokenize text."""
    text = text.lower()  # Lowercase the text
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove punctuation
    return text.split()

def get_bigrams(words):
    """Generate bigrams from a list of words."""
    return zip(words, words[1:])

def calculate_bigram_frequencies(essays):
    """Calculate bigram frequencies in a collection of essays."""
    bigrams = []
    for essay in essays:
        words = clean_text(essay)
        bigrams.extend(get_bigrams(words))
    bigram_counts = Counter(bigrams)
    total_bigrams = sum(bigram_counts.values())
    bigram_freq = {bigram: count / total_bigrams for bigram, count in bigram_counts.items()}
    return bigram_freq

# Load dataset
path = 'megaset4.csv'  # Update with your actual path
path = 'train_essays.csv'
data = pd.read_csv(path)

# Split the data into training and testing sets
train_data, _ = train_test_split(data, test_size=0.00001, random_state=42)

# Ensure essays are strings and not empty or NaN
train_data.dropna(subset=['text'], inplace=True)
train_data['text'] = train_data['text'].astype(str)

# Process AI and Human Essays

ai_bigram_freq = calculate_bigram_frequencies(train_data[train_data['generated'] == 1]['text'])
print("Finished AI bigram freq")
human_bigram_freq = calculate_bigram_frequencies(train_data[train_data['generated'] == 0]['text'])
print("Finished Human bigram freq")

# Prepare the combined DataFrame
bigram_frequencies_data = []
for bigram in set(ai_bigram_freq.keys()).union(set(human_bigram_freq.keys())):
    ai_freq = ai_bigram_freq.get(bigram, 0)
    human_freq = human_bigram_freq.get(bigram, 0)
    prevalence_factor = 0
    if human_freq > 0 and ai_freq > 0:
        prevalence_factor = ai_freq / human_freq if ai_freq >= human_freq else -human_freq / ai_freq
        bigram_frequencies_data.append({
            'Bigram': ' '.join(bigram),
            'Prevalence_Factor': prevalence_factor
        })

# Create DataFrame
bigram_frequencies_df = pd.DataFrame(bigram_frequencies_data)

# Sort by 'Prevalence_Factor'
bigram_frequencies_df.sort_values(by='Prevalence_Factor', ascending=False, inplace=True)

# Write to CSV
bigram_frequencies_df.to_csv('two_word_prevalence_table2.csv', index=False)




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import re

def clean_text(text):
    """Clean and tokenize text."""
    text = text.lower()  # Lowercase the text
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove punctuation
    return text.split()

def get_trigrams(words):
    """Generate trigrams from a list of words."""
    return zip(words, words[1:], words[2:])

def calculate_trigram_frequencies(essays):
    """Calculate trigram frequencies in a collection of essays."""
    trigrams = []
    for essay in essays:
        words = clean_text(essay)
        trigrams.extend(get_trigrams(words))
    trigram_counts = Counter(trigrams)
    total_trigrams = sum(trigram_counts.values())
    trigram_freq = {trigram: count / total_trigrams for trigram, count in trigram_counts.items()}
    return trigram_freq

# Load dataset
path = 'megaset4.csv'  # Update with your actual path
path = 'train_essays.csv'
data = pd.read_csv(path)

# Split the data into training and testing sets
train_data, _ = train_test_split(data, test_size=0.00001, random_state=42)

# Ensure essays are strings and not empty or NaN
train_data.dropna(subset=['text'], inplace=True)
train_data['text'] = train_data['text'].astype(str)

# Process AI and Human Essays
ai_trigram_freq = calculate_trigram_frequencies(train_data[train_data['generated'] == 1]['text'])
print("Finished AI bigram freq")
human_trigram_freq = calculate_trigram_frequencies(train_data[train_data['generated'] == 0]['text'])
print("Finished Human bigram freq")

# Prepare the combined DataFrame
trigram_frequencies_data = []
for trigram in set(ai_trigram_freq.keys()).union(set(human_trigram_freq.keys())):
    ai_freq = ai_trigram_freq.get(trigram, 0)
    human_freq = human_trigram_freq.get(trigram, 0)
    prevalence_factor = 0
    if human_freq > 0 and ai_freq > 0:
        prevalence_factor = ai_freq / human_freq if ai_freq >= human_freq else -human_freq / ai_freq
        trigram_frequencies_data.append({
            'Trigram': ' '.join(trigram),
            'Prevalence_Factor': prevalence_factor
        })

# Create DataFrame
trigram_frequencies_df = pd.DataFrame(trigram_frequencies_data)

# Sort by 'Prevalence_Factor'
trigram_frequencies_df.sort_values(by='Prevalence_Factor', ascending=False, inplace=True)

# Write to CSV
trigram_frequencies_df.to_csv('three_word_prevalence_table2.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import re

def clean_text(text):
    """Clean and tokenize text."""
    text = text.lower()  # Lowercase the text
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove punctuation
    return text.split()

def get_fourgrams(words):
    """Generate fourgrams from a list of words."""
    return zip(words, words[1:], words[2:], words[3:])

def calculate_fourgram_frequencies(essays):
    """Calculate fourgram frequencies in a collection of essays."""
    fourgrams = []
    for essay in essays:
        words = clean_text(essay)
        fourgrams.extend(get_fourgrams(words))
    fourgram_counts = Counter(fourgrams)
    total_fourgrams = sum(fourgram_counts.values())
    fourgram_freq = {fourgram: count / total_fourgrams for fourgram, count in fourgram_counts.items()}
    return fourgram_freq

# Load dataset
path = 'megaset4.csv'  # Update with your actual path
path = 'train_essays.csv'
data = pd.read_csv(path)

# Split the data into training and testing sets
train_data, _ = train_test_split(data, test_size=0.00001, random_state=42)

# Ensure essays are strings and not empty or NaN
train_data.dropna(subset=['text'], inplace=True)
train_data['text'] = train_data['text'].astype(str)

# Process AI and Human Essays
ai_fourgram_freq = calculate_fourgram_frequencies(train_data[train_data['generated'] == 1]['text'])
human_fourgram_freq = calculate_fourgram_frequencies(train_data[train_data['generated'] == 0]['text'])

# Prepare the combined DataFrame
fourgram_frequencies_data = []
for fourgram in set(ai_fourgram_freq.keys()).union(set(human_fourgram_freq.keys())):
    ai_freq = ai_fourgram_freq.get(fourgram, 0)
    human_freq = human_fourgram_freq.get(fourgram, 0)
    prevalence_factor = 0
    if human_freq > 0 and ai_freq > 0:
        prevalence_factor = ai_freq / human_freq if ai_freq >= human_freq else -human_freq / ai_freq
        fourgram_frequencies_data.append({
            'Fourgram': ' '.join(fourgram),
            'Prevalence_Factor': prevalence_factor
        })

# Create DataFrame
fourgram_frequencies_df = pd.DataFrame(fourgram_frequencies_data)

# Sort by 'Prevalence_Factor'
fourgram_frequencies_df.sort_values(by='Prevalence_Factor', ascending=False, inplace=True)

# Write to CSV
fourgram_frequencies_df.to_csv('four_word_prevalence_table.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import re

def clean_text(text):
    """Clean and tokenize text."""
    text = text.lower()  # Lowercase the text
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove punctuation
    return text.split()

def get_fivegrams(words):
    """Generate fivegrams from a list of words."""
    return zip(words, words[1:], words[2:], words[3:], words[4:])

def calculate_fivegram_frequencies(essays):
    """Calculate fivegram frequencies in a collection of essays."""
    fivegrams = []
    for essay in essays:
        words = clean_text(essay)
        fivegrams.extend(get_fivegrams(words))
    fivegram_counts = Counter(fivegrams)
    total_fivegrams = sum(fivegram_counts.values())
    fivegram_freq = {fivegram: count / total_fivegrams for fivegram, count in fivegram_counts.items()}
    return fivegram_freq

# Load dataset
path = 'megaset4.csv'  # Update with your actual path
data = pd.read_csv(path)

# Split the data into training and testing sets
train_data, _ = train_test_split(data, test_size=0.00001, random_state=42)

# Ensure essays are strings and not empty or NaN
train_data.dropna(subset=['text'], inplace=True)
train_data['text'] = train_data['text'].astype(str)

# Process AI and Human Essays
ai_fivegram_freq = calculate_fivegram_frequencies(train_data[train_data['generated'] == 1]['text'])
human_fivegram_freq = calculate_fivegram_frequencies(train_data[train_data['generated'] == 0]['text'])

# Prepare the combined DataFrame
fivegram_frequencies_data = []
for fivegram in set(ai_fivegram_freq.keys()).union(set(human_fivegram_freq.keys())):
    ai_freq = ai_fivegram_freq.get(fivegram, 0)
    human_freq = human_fivegram_freq.get(fivegram, 0)
    prevalence_factor = 0
    if human_freq > 0 and ai_freq > 0:
        prevalence_factor = ai_freq / human_freq if ai_freq >= human_freq else -human_freq / ai_freq
        fivegram_frequencies_data.append({
            'Fivegram': ' '.join(fivegram),
            'Prevalence_Factor': prevalence_factor
        })

# Create DataFrame
fivegram_frequencies_df = pd.DataFrame(fivegram_frequencies_data)

# Sort by 'Prevalence_Factor'
fivegram_frequencies_df.sort_values(by='Prevalence_Factor', ascending=False, inplace=True)

# Write to CSV
fivegram_frequencies_df.to_csv('five_word_prevalence_table.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import re
import numpy as np

# Define function to clean text
def clean_text(text):
    """Clean and tokenize text."""
    text = text.lower()  # Lowercase the text
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove punctuation
    return text.split()

# Calculate word frequencies in essays
def calculate_word_frequencies(essays):
    """Calculate word frequencies in a collection of essays."""
    words = []
    for essay in essays:
        words.extend(clean_text(essay))
    word_counts = Counter(words)
    total_words = sum(word_counts.values())
    word_freq = {word: count / total_words for word, count in word_counts.items()}
    return word_freq

# Load dataset
path = 'megaset4.csv'  # Update with your actual path
data = pd.read_csv(path)

# Split data based on readability into 5 groups
data['readability_group'] = pd.qcut(data['readability'], 5, labels=range(1, 6))

print(data.groupby('readability_group')['readability'].apply(lambda x: (x.min(), x.max())).to_dict())


# Loop through each readability group to calculate and save weighted prevalences
for group in range(1, 6):
    group_data = data[data['readability_group'] == group]
    
    # Split the group data into AI and Human subsets
    ai_data = group_data[group_data['generated'] == 1]['text']
    human_data = group_data[group_data['generated'] == 0]['text']
    
    # Calculate word frequencies for AI and Human essays
    ai_word_freq = calculate_word_frequencies(ai_data)
    human_word_freq = calculate_word_frequencies(human_data)
    
    # Prepare the combined DataFrame for prevalence factors
    prevalence_data = []
    for word in set(ai_word_freq.keys()).union(set(human_word_freq.keys())):
        ai_freq = ai_word_freq.get(word, 0)
        human_freq = human_word_freq.get(word, 0)
        if human_freq > 0 and ai_freq > 0:  # Ensure word exists in both corpuses
            prevalence_factor = ai_freq / human_freq if ai_freq > human_freq else -human_freq / ai_freq
            prevalence_data.append({
            'word': word,
            'prevalence': prevalence_factor
        })
    # Create DataFrame and sort by 'prevalence'
    prevalence_df = pd.DataFrame(prevalence_data)
    prevalence_df.sort_values(by='prevalence', ascending=False, inplace=True)
    
    # Save to CSV
    prevalence_df.to_csv(f'weighted_prevalence{group}.csv', index=False)

print("Weighted prevalence tables created.")


In [ ]:
import numpy as np
import complex

prevalence_table = word_frequencies_df
prevalence_dict = prevalence_table.set_index('Word')['Prevalence_Factor'].to_dict()

# test_data = pd.read_csv('Essays/Training_Essay_Data.csv')
# test_data = pd.read_csv('undetectable.csv')
test_data = pd.read_csv('megaset.csv').sample(10000)
test_data = pd.read_csv('human_expert_essays.csv')
test_data = pd.read_csv('human.csv')
test_data = pd.read_csv('train_v2_drct_02.csv').sample(10000)
test_data = pd.read_csv('metadataset.csv').sample(10000)
test_data = pd.read_csv('final_test2.csv').sample(10000)


prev = []

def scan_text_for_ai(text, print2=False):
    # Initialize the prevalence score
    prevalence_score = 0
    words = text.lower().split()
    
    # Calculate Prevalence for each word in the text using the dictionary for fast lookup
    for word in words:
        word = clean_word(word)
        # check if the word is in the prevalence dictionary
        if word in prevalence_dict:
            prevalence = prevalence_dict.get(word, 0)
        else:
            prevalence = 0

        if (prevalence > 0):
            prevalence -= 1
        else:
            prevalence += 1
        prevalence_score += prevalence
    prev.append(prevalence_score)
    
    if print2:
        print(prevalence_score)

    # verdict2 = predict_ai(text)

    # if (verdict2 == 0):
    #      return 0
    # if (verdict2 == 1):
    #     return 1
   
    # Calculate Verdict based on the total prevalence score
    if prevalence_score <= 0:
        verdict =  0  #1 / prevalence_score  
    else:
        verdict =  1 #1 - 1 / prevalence_score
    
    return verdict



# Load the dataset
# Apply the function to each essay to get a verdict
test_data['Verdict'] = test_data['text'].apply(scan_text_for_ai)

# Round the 'Verdict' values
test_data['Verdict'] = test_data['Verdict'].apply(round)

# You can now safely create the 'Prediction' column
test_data['Prediction'] = test_data['Verdict']

# generated_column = 'label'
generated_column = 'generated'
# Calculate overall accuracy and error rates
accuracy = (test_data['Prediction'] == test_data[generated_column]).mean()
fp = test_data[(test_data['Prediction'] == 1) & (test_data[generated_column] == 0)].shape[0]
fn = test_data[(test_data['Prediction'] == 0) & (test_data[generated_column] == 1)].shape[0]
tp = test_data[(test_data['Prediction'] == 1) & (test_data[generated_column] == 1)].shape[0]
tn = test_data[(test_data['Prediction'] == 0) & (test_data[generated_column] == 0)].shape[0]

digits = 8
accuracy = round(100 * accuracy, digits)


fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
fpr = round(100 * fpr, digits)
fnr = round(100 * fnr, digits)
                                   

def mean(listx):
    total = 0
    for i in range(len(listx)):
        total += listx[i]
    return total / len(listx)
    

print(f"Accuracy: {accuracy}%")
print(f"False Positive Rate (FPR): {fpr}%")
print(f"False Negative Rate (FNR): {fnr}%")
print(f"Average prevalence = {mean(prev)}%")
print(f"Prevalence std = {np.std(prev)}%")

 

In [ ]:
with open('input.txt', 'r') as file:
    text = file.read()
verdict = scan_text_for_ai(text, True)
print(verdict)
print(text)